In [0]:
df = spark.read.table('silver.movie')

In [0]:
df.show()

+---+---------+--------------------+----------+--------------------+------+---------+--------------------+--------+--------------------+--------------------+----------+--------+---------+
| id|   imdbId|               title|   release|                plot|rating|voteCount|             tagline|language|           companies|           countries|popularity|  budget|  revenue|
+---+---------+--------------------+----------+--------------------+------+---------+--------------------+--------+--------------------+--------------------+----------+--------+---------+
|  1|tt0035423|      Kate & Leopold|2002-02-14|An English Duke f...|   6.4|    87814|If they lived in ...|      en|[{'name': 'Mirama...|[{'iso_3166_1': '...|  7.196356|48000000| 76019048|
|  6|tt0053580|           The Alamo|1961-07-14|In 1836, a small ...|   6.8|    17070|The Mission That ...|      en|[{'name': 'United...|[{'iso_3166_1': '...|   4.73816|12000000|  7900000|
| 12|tt0053779|       La Dolce Vita|1960-06-10|A series of s

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when
from pyspark.sql.window import Window

def build_chart(percentile=0.85):
    # Tính trung bình cộng của `vote_average`.
    C = df.agg({'rating': 'mean'}).collect()[0][0]

    # Tính lượng bình chọn tối thiểu mà một bộ phim cần có để được xem xét (ở phần trăm thứ `percentile`).
    m = df.approxQuantile('voteCount', [percentile], 0)[0]
    qualified = df.filter(df['voteCount'] >= m).select(['title', 'voteCount', 'rating'])
    
    # Tính điểm xếp hạng trọng số (`wr`) cho mỗi bộ phim đủ điều kiện.
    qualified = qualified.withColumn('weighted rating', (col('voteCount')/(col('voteCount')+m) * col('rating')) + (m/(m+col('voteCount')) * C))

    # Xếp hạng các bộ phim dựa trên `wr` và chỉ giữ lại 10 bộ phim hàng đầu.
    window = Window.orderBy(col('weighted rating').desc())
    qualified = qualified.withColumn('rank', F.rank().over(window)).filter(col('rank') <= 10)
    
    return qualified

In [0]:
build_chart(0.9).show()

+--------------------+---------+------+-----------------+----+
|               title|voteCount|rating|  weighted rating|rank|
+--------------------+---------+------+-----------------+----+
|    Band of Brothers|   502155|   9.4|9.335279685195923|   1|
|     Planet Earth II|   153861|   9.5|9.291528478702752|   2|
|The Shawshank Red...|  2812263|   9.3|9.288604444207381|   3|
|        Planet Earth|   216693|   9.4|9.253669719365293|   4|
|       The Godfather|  1959950|   9.2| 9.18416532035472|   5|
|            Sherlock|   971143|   9.1|9.069189960532354|   6|
|     The Dark Knight|  2793959|   9.0| 8.98956701647809|   7|
|The Lord of the R...|  1925558|   9.0|  8.9848854414852|   8|
|    Schindler's List|  1413745|   9.0|8.979450825852933|   9|
|The Godfather Par...|  1330803|   9.0|8.978179362047717|  10|
+--------------------+---------+------+-----------------+----+

